<a href="https://colab.research.google.com/github/LoliKillers/katacoda-scenarios/blob/master/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Colab RDP** : Desktop Jarak Jauh ke Instance Colab
> **Peringatan : Bukan untuk Penambangan Cryptocurrency<br></br>** >**Mengapa sumber daya perangkat keras seperti GPU T4 tidak tersedia untuk saya?** Perangkat keras terbaik yang tersedia diprioritaskan untuk pengguna yang menggunakan Colaboratory secara interaktif daripada untuk komputasi yang berjalan lama. Pengguna yang menggunakan Colaboratory untuk komputasi yang berjalan lama mungkin akan dibatasi sementara dalam jenis perangkat keras yang disediakan untuk mereka, dan/atau durasi penggunaan perangkat keras tersebut. Kami mendorong pengguna dengan kebutuhan komputasi tinggi untuk menggunakan UI Colaboratory dengan runtime lokal. Harap diperhatikan bahwa penggunaan Colaboratory untuk penambangan mata uang kripto dilarang sepenuhnya, dan dapat mengakibatkan larangan menggunakan Colab sama sekali. Google Colab dapat memberi Anda Instance dengan RAM dan GPU 12 GB selama 12 jam (Maks.) untuk pengguna Gratis. Siapa pun dapat menggunakannya untuk melakukan Tugas Berat. Untuk menggunakan Notebook serupa lainnya, gunakan Repositori saya **[Colab Loli](https://github.com/LoliKillers/Colab-Loli)**


In [ ]:
#@title **Buat User**
#@markdown Masukkan Username dan Password

import os

username = "user" #@param {type:"string"}
password = "root" #@param {type:"string"}

print("Membuat User dan mengaturnya")

# Membuat user
os.system(f"useradd -m {username}")

# Menambahkan user ke pengguna group sudo
os.system(f"adduser {username} sudo")
    
# Membuat password untuk pengguna 'root'
os.system(f"echo '{username}:{password}' | sudo chpasswd")

# Ubah shell default dari sh ke bash
os.system("sed -i 's/\/bin\/sh/\/bin\/bash/g' /etc/passwd")

print("User success di buat dan di konfigurasi")


In [ ]:
#@title **RDP**
#@markdown  Membutuhkan waktu 4-5 menit untuk installation

import os
import subprocess

#@markdown  Kunjungi http://remotedesktop.google.com/headless dan Salin command debian linux setelah otentikasi

CRP = "" #@param {type:"string"}

#@markdown Masukkan pin 6 digit atau lebih
Pin = 123456 #@param {type: "integer"}


class CRD:
    def __init__(self):
        os.system("apt update")
        self.installCRD()
        self.installDesktopEnvironment()
        self.installGoogleChorme()
        self.finish()

    @staticmethod
    def installCRD():
        print("Memasang Chrome Remote Desktop")
        subprocess.run(['wget', 'https://dl.google.com/linux/direct/chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['dpkg', '--install', 'chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)

    @staticmethod
    def installDesktopEnvironment():
        print("Memasang Desktop Environment")
        os.system("export DEBIAN_FRONTEND=noninteractive")
        os.system("apt install --assume-yes xfce4 desktop-base xfce4-terminal")
        os.system("bash -c 'echo \"exec /etc/X11/Xsession /usr/bin/xfce4-session\" > /etc/chrome-remote-desktop-session'")
        os.system("apt remove --assume-yes gnome-terminal")
        os.system("apt install --assume-yes xscreensaver")
        os.system("systemctl disable lightdm.service")

    @staticmethod
    def installGoogleChorme():
        print("Memasang Google Chrome")
        subprocess.run(["wget", "https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(["dpkg", "--install", "google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)

    @staticmethod
    def finish():
        print("Penyelesaian")
        os.system(f"adduser {username} chrome-remote-desktop")
        command = f"{CRP} --pin={Pin}"
        os.system(f"su - {username} -c '{command}'")
        os.system("service chrome-remote-desktop start")
        print("Success, semua sudah dibuat dan silahkan buka Chrome Remote Desktop")


try:
    if username:
        if CRP == "":
            print("Silakan masukkan kode auth dari tautan yang saya berikan")
        elif len(str(Pin)) < 6:
            print("Harap massukkan pin paling minim 6 digit")
        else:
            CRD()
except NameError as e:
    print("Error : variable username kosong")
    print("Buat user terlebih dahulu!")


In [ ]:
#@title **Google Drive Mount**
#@markdown Google Drive digunakan sebagai HDD Persistensi untuk file.<br>
#@markdown Dipasang di direktori Home `user` di dalam folder drive
#@markdown (Jika variabel `username` tidak di isi maka gunakan root sebagai default).


def MountGDrive():
    from os import environ as env
    from google.colab import drive

    config = env['CLOUDSDK_CONFIG']
    addr = env['TBE_CREDS_ADDR']

    ! runuser -l $user -c "yes | python3 -m pip install --user google-colab"  > /dev/null 2>&1

    mount = """from os import environ as env
from google.colab import drive

env['CLOUDSDK_CONFIG']  = '{config}'
env['TBE_CREDS_ADDR'] = '{addr}'

drive.mount('{mountpoint}')""".format(config=config, addr=addr, mountpoint=mountpoint)

    with open('/content/mount.py', 'w') as script:
        script.write(mount)

    ! runuser -l $user -c "python3 /content/mount.py"

try:
    mountpoint = f"/home/{username}/drive"
    user = username
except NameError:
    print("Variable `username` kosong, pemasangan di `/content/drive' menggunakan `root'")
    mountpoint = '/content/drive'
    user = 'root'

MountGDrive()


In [ ]:
#@title **SSH**

! pip install colab_ssh --upgrade &> /dev/null

Ngrok = False #@param {type:'boolean'}
Agro = False #@param {type:'boolean'}


#@markdown Copy authtoken dari https://dashboard.ngrok.com/auth (harus dari ngrok)
ngrokToken = "" #@param {type:'string'}


def runNGROK():
    from colab_ssh import launch_ssh
    from IPython.display import clear_output
    launch_ssh(ngrokToken, password)
    clear_output()

    print("ssh", username, end='@')
    ! curl -s http://localhost:4040/api/tunnels | python3 -c \
            "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'][6:].replace(':', ' -p '))"


def runAgro():
    from colab_ssh import launch_ssh_cloudflared
    launch_ssh_cloudflared(password=password)


try:
    if username:
        pass
    elif password:
        pass
except NameError:
    print("User tidak di temukan, menggunakan username dan password dengan 'root'")
    username='root'
    password='root'


if Agro and Ngrok:
    print("Anda tidak bisa memilih semuanya")
    print("Pilih hanya salah satu dari mereka")
elif Agro:
    runAgro()
elif Ngrok:
    if ngrokToken == "":
        print("Ngrok authtoken tidak ditemukan, harap masukkan dengan benar")
    else:
        runNGROK()
else:
    print("Pilih salah satu saja")


In [ ]:
#@title **Colab Shutdown**

#@markdown untuk mematikan NGROK
NGROK = False #@param {type:'boolean'}

#@markdown Untuk Melepas GDrive
GDrive = False #@param {type:'boolean'}

#@markdown Untuk Membuat colab Istirahat
Sleep = True #@param {type:'boolean'}

if NGROK:
    ! killall ngrok

if GDrive:
    with open('/content/unmount.py', 'w') as unmount:
        unmount.write("""from google.colab import drive
drive.flush_and_unmount()""")
    
    try:
        if user:
            ! runuser $user -c 'python3 /content/unmount.py'
    except NameError:
        print("Tidak terpasang dengan google drive")

if Sleep:
    from time import sleep
    sleep(43200)